## Question-4: Testing on Manually annotated dataset

In [129]:
from transformers import AutoModelForTokenClassification,AutoTokenizer

# Load the fine-tuned model
model = AutoModelForTokenClassification.from_pretrained('IndicNER_model')

tokenizer=AutoTokenizer.from_pretrained('IndicNER_model')

In [130]:
# Get the number of trainable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

num_parameters = count_parameters(model)
print("Number of trainable parameters:", num_parameters)

Number of trainable parameters: 166771207


In [131]:
import torch
def get_predictions( sentence, tokenizer, model ):
  # Let us first tokenize the sentence - split words into subwords
  tok_sentence = tokenizer(sentence, return_tensors='pt')

  with torch.no_grad():
    # we will send the tokenized sentence to the model to get predictions
    logits = model(**tok_sentence).logits.argmax(-1)

    # We will map the maximum predicted class id with the class label
    predicted_tokens_classes = [model.config.id2label[t.item()] for t in logits[0]]

    predicted_labels = []

    previous_token_id = 0
    # we need to assign the named entity label to the head word and not the following sub-words
    word_ids = tok_sentence.word_ids()
    for word_index in range(len(word_ids)):
        if word_ids[word_index] == None:
            previous_token_id = word_ids[word_index]
        elif word_ids[word_index] == previous_token_id:
            previous_token_id = word_ids[word_index]
        else:
            predicted_labels.append( predicted_tokens_classes[ word_index ] )
            previous_token_id = word_ids[word_index]

    return predicted_labels

In [104]:
# Open the text file in read mode
with open('25 Sentences.txt', 'r') as file:
  sentences=file.read().split('$')

In [134]:
label_id={ "LABEL_0":'O', "LABEL_1": "B-PER", "LABEL_2": "I-PER", "LABEL_3": "B-ORG", "LABEL_4":"I-ORG", "LABEL_5":"B-LOC", "LABEL_6":"I-LOC"}
for sentence in sentences:
  predicted_labels = get_predictions(sentence=sentence,
                                   tokenizer=tokenizer,
                                   model=model
                                   )

  words = sentence.split(' ')
  # Ensure the length of predicted labels matches the number of words
  if len(predicted_labels) > len(words):
    predicted_labels = predicted_labels[:len(words)]

  for i in range(len(words)):
    print( sentence.split(' ')[i] + '\t' + predicted_labels[i])

నా	I-LOC
ప్రయాణం	I-LOC
మొత్తం	O
పదిరోజులే	O
కాబట్టి	I-LOC
నేను	I-LOC
మూడు	O
నాలుగు	I-LOC
ప్రదేశాల	I-LOC
కన్న	I-LOC
ఎన్నుకోలేను	O
,	O
ఒక్కో	I-LOC
చోటా	I-LOC
రెండు	I-LOC
మూడు	I-LOC
రోజుల	I-LOC
కన్న	I-LOC
ఎక్కువ	I-LOC
సమయం	I-LOC
ఉండలేను	I-LOC
.	I-LOC
ప్రజల	I-LOC
మనోభావాలు	I-LOC
,	I-LOC
ఎమోషన్స్	O
కి	O
అంతగా	I-LOC
ప్రాముఖ్యత	O
ఇవ్వకుండా	O
తానుఅనుకున్న	O
పనులు	O
చక్కబెట్టేవారు	O
.	O
మద్యం	I-LOC
లేదా	I-LOC
సాధ్యం	I-LOC
కిరోసిన్	I-LOC
ఉపయోగించడం	I-LOC
ద్వారా	I-LOC
మరింత	I-LOC
సమర్ధవంతంగా	I-LOC
జరిమానా	O
నష్టం	I-LOC
తొలగించడానికి	I-LOC
.	I-LOC
మరో	I-LOC
వివాహం	I-LOC
చేసుకునేందుకు	I-LOC
వీలుగా	I-LOC
భార్య	I-LOC
పాయల్‌	I-LOC
నుంచి	I-LOC
విడాకులు	I-LOC
ఇప్పించాలి	I-LOC
'	I-LOC
అని	I-LOC
ఒమర్	O
పిటిషన్‌లో	I-LOC
విన్నవించారు	I-LOC
.	I-LOC
అన్ని	I-LOC
ఇతర	I-LOC
ఈస్ట్	I-LOC
వంటి	I-LOC
,	I-LOC
ఈ	I-LOC
జాతులు	I-LOC
చాలా	I-LOC
విటమిన్లు	I-LOC
మరియు	I-LOC
ట్రేస్	O
ఎలిమెంట్స్	I-LOC
చాలా	I-LOC
గొప్ప	I-LOC
ఉంది	I-LOC
,	I-LOC
ప్రోటీన్	O
చాలా	I-LOC
కలిగి	I-LOC
మరియు	I-LOC
మొత్తం	I-LOC
చాలా	I-LOC
విలువైన	I-LOC

In [136]:
# Open a text file in write mode
with open('predicted_labels_bert.txt', 'w') as file:
  for sentence in sentences:
    predicted_labels = get_predictions(sentence=sentence,
                                      tokenizer=tokenizer,
                                      model=model)
    print(len(sentence.split(' ')),len(predicted_labels))
    # Ensure the length of predicted labels matches the number of words
    if len(predicted_labels) > len(words):
      predicted_labels = predicted_labels[:len(words)]
    for index in range(len(sentence.split(' '))):
      if (sentence.split(' ')[index]!=" "):
        #print(len(sentence.split(' ')),len())
        #print( sentence.split(' ')[index] + '\t' +  label_id[predicted_labels[index]])
        file.write(sentence.split(' ')[index] + '\t' + predicted_labels[index] + '\n')

22 22
12 12
12 12
15 15
26 26
144 158
13 13
13 13
24 24
16 16
11 11
24 24
43 46
27 27
16 16
11 11
14 14
13 13
25 25
19 20
16 16
32 32
15 16
14 14
14 14


In [137]:
def read_and_tokenize_file(file_path, tokenizer):
    data = []
    with open(file_path, "r", encoding="utf-8") as file:
        for line in file:
            line = line.strip()
            if line:
                line_data = eval(line)  # Convert the string representation of the dictionary to a dictionary
                tokens = line_data["tokens"]
                ner_tags = line_data["ner_tags"]
                line_dict = {"tokens": tokens, "ner_tags": ner_tags}
                data.append(line_dict)
    return data


# Specify the path to the text file
file_path = "colab_maual.txt"

# Read and tokenize the file
tokenized_data = read_and_tokenize_file(file_path, tokenizer)
ground_truth_tokens=[]
ground_truth_tokenized_labels=[]
# Print the tokenized data
c=0
for line_data in tokenized_data:
  # c+=1
  # if(c==4):
  #   for i in range(15):
  #     print(line_data["tokens"][i],line_data["ner_tags"][i])
  if("," in line_data["ner_tags"]):
    for i in range(len(line_data["ner_tags"])):
      print(line_data["tokens"][i],line_data["ner_tags"][i])
      if(line_data["ner_tags"][i]=="OO"): print("culprit ",i)
  ground_truth_tokens.extend(line_data["tokens"])
  ground_truth_tokenized_labels.extend(line_data["ner_tags"])
  #print(len(line_data["tokens"]),len(line_data["ner_tags"]))

In [94]:
label_id={ "LABEL_0":'O', "LABEL_1": "B-PER", "LABEL_2": "I-PER", "LABEL_3": "B-ORG", "LABEL_4":"I-ORG", "LABEL_5":"B-LOC", "LABEL_6":"I-LOC"}

In [ ]:
import os
print(os.getcwd())  # Print current working directory

/kaggle/working


In [138]:
label_id={ "LABEL_0":'O', "LABEL_1": "B-PER", "LABEL_2": "I-PER", "LABEL_3": "B-ORG", "LABEL_4":"I-ORG", "LABEL_5":"B-LOC", "LABEL_6":"I-LOC"}
import torch
predicted_labels_global=[]
predicted_tokens=[]
for sentence in sentences:
  predicted_labels = get_predictions(sentence=sentence,
                                    tokenizer=tokenizer,
                                    model=model)

  words = sentence.split(' ')
  # Ensure the length of predicted labels matches the number of words
  if len(predicted_labels) > len(words):
    predicted_labels = predicted_labels[:len(words)]
  #print(len(words),len(predicted_labels))
  for i in range(len(words)):
    #print(words[i], predicted_labels[i])
    predicted_tokens.append(words[i])
    predicted_labels_global.append( predicted_labels[i])
  # for index in range(len(sentence.split(' '))):
  #   if (sentence.split(' ')[index]!=" "):
  #     #print( sentence.split(' ')[index] + '\t' + predicted_labels[index] )
  #     predicted_tokens.append(sentence.split(' ')[index])
  #     predicted_labels_global.append( label_id[predicted_labels[index]])
print()

22 22
12 12
12 12
15 15
26 26
144 144
13 13
13 13
24 24
16 16
11 11
24 24
43 43
27 27
16 16
11 11
14 14
13 13
25 25
19 19
16 16
32 32
15 15
14 14
14 14



In [140]:
print(len(predicted_labels_global),len(ground_truth_tokenized_labels))

591 591


In [141]:
print(len(ground_truth_tokens),len(predicted_tokens))

591 591


In [142]:
from sklearn.metrics import precision_score, recall_score, f1_score,classification_report
def calculate_metrics(predicted_labels, ground_truth_labels):
    precision = precision_score(ground_truth_labels, predicted_labels, average='macro')
    recall = recall_score(ground_truth_labels, predicted_labels, average='macro')
    f1_sco = f1_score(ground_truth_labels, predicted_labels, average='macro')
    #report=classification_report(ground_truth_labels, predicted_labels)
    return precision, recall, f1_sco

In [143]:
p,r,f1=calculate_metrics(predicted_labels_global,ground_truth_tokenized_labels)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [144]:
print("Precision: ",p,"\n","Recall: ",r,"\n","F1-Score: ",f1)

Precision:  0.15515468827881657 
 Recall:  0.19554390826756632 
 F1-Score:  0.08705967654358196


In [145]:
report=classification_report(ground_truth_tokenized_labels,predicted_labels_global)
print(report)

              precision    recall  f1-score   support

       B-LOC       0.00      0.00      0.00         6
      B-MISC       0.00      0.00      0.00        27
       B-ORG       0.00      0.00      0.00         5
       B-PER       0.00      0.00      0.00        19
       I-LOC       0.00      1.00      0.00         1
      I-MISC       0.00      0.00      0.00        15
       I-ORG       0.29      0.50      0.36         4
       I-PER       0.25      0.09      0.13        11
           O       0.86      0.17      0.28       503

    accuracy                           0.15       591
   macro avg       0.16      0.20      0.09       591
weighted avg       0.74      0.15      0.25       591



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
from sklearn.metrics import classification_report
import numpy as np

def custom_classification_report(y_true, y_pred, labels=None, target_names=None):
    report = classification_report(y_true, y_pred, labels=labels, target_names=target_names, output_dict=True)

    # Exclude class labels with support = 0
    non_zero_labels = [label for label, stats in report.items() if label != 'accuracy' and stats['support'] != 0]

    # Calculate macro avg excluding classes with support = 0
    macro_precision = np.mean([report[label]['precision'] for label in non_zero_labels])
    macro_recall = np.mean([report[label]['recall'] for label in non_zero_labels])
    macro_f1 = np.mean([report[label]['f1-score'] for label in non_zero_labels])

    # Update macro avg in the report
    report['macro avg'] = {'precision': macro_precision, 'recall': macro_recall, 'f1-score': macro_f1, 'support': sum(report[label]['support'] for label in non_zero_labels)}

    # Remove class labels with support = 0 from the report
    for label in report.copy():
        if label != 'accuracy' and report[label]['support'] == 0:
            del report[label]

    return report


report = custom_classification_report(ground_truth_tokenized_labels,predicted_labels_global)

# Print the custom classification report
print(report)


{',': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1}, 'B-LOC': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 6}, 'B-MISC': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 25}, 'B-ORG': {'precision': 0.6666666666666666, 'recall': 0.4, 'f1-score': 0.5, 'support': 5}, 'B-PER': {'precision': 0.2, 'recall': 0.15789473684210525, 'f1-score': 0.17647058823529413, 'support': 19}, 'I-LOC': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1}, 'I-MISC': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 16}, 'I-ORG': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 4}, 'I-PER': {'precision': 0.2727272727272727, 'recall': 0.2727272727272727, 'f1-score': 0.2727272727272727, 'support': 11}, 'O': {'precision': 0.8695652173913043, 'recall': 0.96, 'f1-score': 0.9125475285171103, 'support': 500}, 'O,': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1}, 'OO': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
